Exercises from _Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow_

**1. Is it okay to initialize all the weights to the same value as long as that value is
selected randomly using He initialization?**

Since we want to avoid symmetries by breaking them, initializing all values to the same will make 
all the weights to be the same, thus, making impossible to break the symmetry.

**2. Is it okay to initialize the bias terms to 0?**

Yes, it is okay, it doest not make much difference.

**3. Name three advantages of the SELU activation function over ReLU.**

It can take negative values, alliviating the vanishing gradients problem.
It has a nonzero derivative, avoiding dying units.
It is smooth everywhere, since ReLU jumps from 0 to 1 at given point.

**4. In which cases would you want to use each of the following activation functions:
ELU, leaky ReLU (and its variants), ReLU, tanh, logistic, and softmax?**

ELU, leaky ReLU: If you need the neural network to be as fast as possible.

ReLU: Autoencoders.

tanh: In a output layer if a number between -1 and 1 is needed.

logistic: In the output layer to estimate a probability.

softmax: In the output layer for probabilities that are mutually exclusive classes.

**5. What may happen if you set the momentum hyperparameter too close to 1 (e.g.,
0.99999) when using an SGD optimizer?**

The algorithm will pick up a lot of speed and it will shoot right past the minimum. It will
make it several times before converging, resulting in a slower training time.

**6. Name three ways you can produce a sparse model.**

Zero out tiny weights. Also apply l1 regularization during training, making it more sparse. At last combining
l1 regularization with dual averaging.

**7. Does dropout slow down training? Does it slow down inference (i.e., making
predictions on new instances)? What are about MC dropout?**

Yes, dropout does slow down training, in general roughly by a factor of two.
However, it has no impact on inference since it is only turned on during training.

In [7]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
assert tf.__version__ >= "2.0"

In [22]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)


# Build a DNN with five hidden layers of 100 neurons each, He initialization, and the ELU activation function.
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 activation="elu",
                                 kernel_initializer="he_normal"))

In [ ]:
# Using Adam optimization and early stopping, try training it on MNIST but
# only on digits 0 to 4, as we will use transfer learning for digits 5 to 9 in the
# next exercise. You will need a softmax output layer with five neurons, and as
# always make sure to save checkpoints at regular intervals and save the final
# model so you can reuse it later.

In [23]:
# Loading the dataset
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [24]:
# Output layer
model.add(keras.layers.Dense(10, activation="softmax"))

# Optimizer
optimizer = keras.optimizers.Adam(learning_rate=3e-5)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [25]:
# Early stopping and checkpoint
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_model.h5", save_best_only=True)

run_index = 1 
run_logdir = os.path.join(os.curdir, "my_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [ ]:
%tensorboard --logdir=./my_logs --port=6006

In [26]:
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 5s 3ms/step - loss: 5.1812 - accuracy: 0.1368 - val_loss: 2.3657 - val_accuracy: 0.1692
Epoch 2/100
1407/1407 [==============================] - 4s 3ms/step - loss: 2.2217 - accuracy: 0.1967 - val_loss: 2.1090 - val_accuracy: 0.2214
Epoch 3/100
1407/1407 [==============================] - 5s 3ms/step - loss: 2.0644 - accuracy: 0.2411 - val_loss: 1.9905 - val_accuracy: 0.2658
Epoch 4/100
1407/1407 [==============================] - 4s 3ms/step - loss: 1.9741 - accuracy: 0.2713 - val_loss: 1.9205 - val_accuracy: 0.2954
Epoch 5/100
1407/1407 [==============================] - 5s 3ms/step - loss: 1.8969 - accuracy: 0.3040 - val_loss: 1.8321 - val_accuracy: 0.3304
Epoch 6/100
1407/1407 [==============================] - 5s 3ms/step - loss: 1.8413 - accuracy: 0.3315 - val_loss: 1.8402 - val_accuracy: 0.3246
Epoch 7/100
1407/1407 [==============================] - 5s 4ms/step - loss: 1.7969 - accuracy: 0.3440 - val_loss: 1.7837 - val_ac

In [27]:
model = keras.models.load_model("my_model.h5")
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 2ms/step - loss: 1.5226 - accuracy: 0.4630


[1.5225569009780884, 0.46299999952316284]

In [28]:
# Now try adding Batch Normalization and compare the learning curves: is it
# converging faster than before? Does it produce a better model?
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_bn_model.h5", save_best_only=True)
run_index = 1
run_logdir = os.path.join(os.curdir, "my_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_bn_model.h5")
model.evaluate(X_valid, y_valid)

Epoch 1/100
1407/1407 [==============================] - 29s 16ms/step - loss: 1.8425 - accuracy: 0.3395 - val_loss: 1.6721 - val_accuracy: 0.4078
Epoch 2/100
1407/1407 [==============================] - 22s 16ms/step - loss: 1.6712 - accuracy: 0.4025 - val_loss: 1.5720 - val_accuracy: 0.4352
Epoch 3/100
1407/1407 [==============================] - 22s 16ms/step - loss: 1.5979 - accuracy: 0.4318 - val_loss: 1.5345 - val_accuracy: 0.4482
Epoch 4/100
1407/1407 [==============================] - 23s 16ms/step - loss: 1.5467 - accuracy: 0.4484 - val_loss: 1.4814 - val_accuracy: 0.4720
Epoch 5/100
1407/1407 [==============================] - 26s 18ms/step - loss: 1.5025 - accuracy: 0.4669 - val_loss: 1.4358 - val_accuracy: 0.4880
Epoch 6/100
1407/1407 [==============================] - 23s 16ms/step - loss: 1.4639 - accuracy: 0.4790 - val_loss: 1.4410 - val_accuracy: 0.4818
Epoch 7/100
1407/1407 [==============================] - 22s 16ms/step - loss: 1.4328 - accuracy: 0.4916 - val_loss: 1

[1.3205863237380981, 0.5437999963760376]

In [32]:
# Is the model overfitting the training set? Try adding dropout to every layer
# and try again. Does it help?
session = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1



2022-06-08 12:58:37.931327: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-08 12:58:37.931524: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-08 12:58:37.931674: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-08 12:58:37.931872: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-08 12:58:37.931980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [34]:
tf.debugging.set_log_device_placement(True)

In [35]:
# Try replacing Batch Normalization with SELU, and make the necessary 
# adjustements to ensure the network self-normalizes 
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=7e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_selu_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_logs", "run_selu_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_selu_model.h5")
model.evaluate(X_valid_scaled, y_valid)

Epoch 1/100
1407/1407 [==============================] - 12s 7ms/step - loss: 1.9415 - accuracy: 0.3041 - val_loss: 1.8389 - val_accuracy: 0.3416
Epoch 2/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.7236 - accuracy: 0.3873 - val_loss: 1.6943 - val_accuracy: 0.3898
Epoch 3/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.6223 - accuracy: 0.4295 - val_loss: 1.7451 - val_accuracy: 0.3762
Epoch 4/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.5567 - accuracy: 0.4495 - val_loss: 1.6339 - val_accuracy: 0.4240
Epoch 5/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.4967 - accuracy: 0.4749 - val_loss: 1.5786 - val_accuracy: 0.4428
Epoch 6/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.4483 - accuracy: 0.4936 - val_loss: 1.5303 - val_accuracy: 0.4580
Epoch 7/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.4039 - accuracy: 0.5074 - val_loss: 1.5353 - val_

[1.4811910390853882, 0.47760000824928284]

In [46]:
# Is the model overfitting the training set? Try adding dropout to every layer
# and try again. Does it help?

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))

model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Adam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_alpha_dropout_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_logs", "run_alpha_dropout_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_alpha_dropout_model.h5")
model.evaluate(X_valid_scaled, y_valid)

Epoch 1/100
1407/1407 [==============================] - 6s 4ms/step - loss: 2.1066 - accuracy: 0.2699 - val_loss: 1.8124 - val_accuracy: 0.3394
Epoch 2/100
1407/1407 [==============================] - 4s 3ms/step - loss: 1.7617 - accuracy: 0.3796 - val_loss: 1.6445 - val_accuracy: 0.4180
Epoch 3/100
1407/1407 [==============================] - 4s 3ms/step - loss: 1.6327 - accuracy: 0.4271 - val_loss: 1.5905 - val_accuracy: 0.4272
Epoch 4/100
1407/1407 [==============================] - 5s 3ms/step - loss: 1.5597 - accuracy: 0.4567 - val_loss: 1.5691 - val_accuracy: 0.4472
Epoch 5/100
1407/1407 [==============================] - 5s 3ms/step - loss: 1.4998 - accuracy: 0.4775 - val_loss: 1.5576 - val_accuracy: 0.4562
Epoch 6/100
1407/1407 [==============================] - 5s 3ms/step - loss: 1.4495 - accuracy: 0.4974 - val_loss: 1.4846 - val_accuracy: 0.4798
Epoch 7/100
1407/1407 [==============================] - 5s 3ms/step - loss: 1.4111 - accuracy: 0.5136 - val_loss: 1.5518 - val_ac

[1.473311185836792, 0.4934000074863434]

In [39]:
class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [40]:
mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer
    for layer in model.layers
])

In [41]:
def mc_dropout_predict_probas(mc_model, X, n_samples=10):
    Y_probas = [mc_model.predict(X) for sample in range(n_samples)]
    return np.mean(Y_probas, axis=0)

def mc_dropout_predict_classes(mc_model, X, n_samples=10):
    Y_probas = mc_dropout_predict_probas(mc_model, X, n_samples)
    return np.argmax(Y_probas, axis=1)

In [42]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

y_pred = mc_dropout_predict_classes(mc_model, X_valid_scaled)
accuracy = np.mean(y_pred == y_valid[:, 0])
accuracy

0.4982